어렵다...

일단 실습부터 해보자.

In [1]:
from gensim import corpora
from preprocess import get_tokenizer

corpus_fname = 'data/processed/corrected_ratings_corpus.txt'

/home/inhwan/Python/Pytorch_ev/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
corpus = [sent for sent in open(corpus_fname, 'r')]
corpus[:3]

['어릴때 보고 지금 다시봐도 재밌어요ㅋㅋ\n',
 '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.\n',
 '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.\n']

In [4]:
documents, tokenized_corpus = [], []
tokenizer = get_tokenizer('mecab')

with open(corpus_fname, 'r', encoding='utf-8') as f:
    for document in f:
        tokens = list(set(tokenizer.morphs(document.strip())))
        documents.append(document)
        tokenized_corpus.append(tokens)        

In [11]:
dictionary = corpora.Dictionary(tokenized_corpus)
dictionary[0], dictionary[1], dictionary[2]

('ㅋㅋ', '고', '다시')

In [13]:
corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]

In [29]:
documents[:3]

['어릴때 보고 지금 다시봐도 재밌어요ㅋㅋ\n',
 '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.\n',
 '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.\n']

LDA 모델 훈련을 시켜보자.

In [25]:
from gensim.models import ldamulticore
LDA = ldamulticore.LdaMulticore(corpus, id2word=dictionary, num_topics=30, workers=8)

학습이 끝난 LDA 모델로부터 각 문서별 토픽 정보를 다음과 같이 얻을 수 있다. 여기서 `minimum_probability` 인자는 해당 값 이하의 토픽 분포는 무시한다는 뜻이다.

In [26]:
all_topics = LDA.get_document_topics(corpus, minimum_probability=0.5, per_word_topics=False)

In [28]:
for doc_idx, topic in enumerate(all_topics[:5]):
    print(doc_idx, topic)

0 [(18, 0.9121122)]
1 [(17, 0.9055252)]
2 [(2, 0.94312084)]
3 []
4 [(21, 0.93082196)]


학습이 끝난 결과를 저장하자.

In [30]:
output_fname='data/sentence-embeddings/lda/lda'

with open(output_fname + ".results", 'w') as f:
    for doc_idx, topic in enumerate(all_topics):
        if len(topic) == 1:
            topic_id, prob = topic[0]
            f.writelines(documents[doc_idx].strip() + "\u241E" + ' '.join(tokenized_corpus[doc_idx]) + "\u241E" + str(topic_id) + "\u241E" + str(prob) + "\n")
LDA.save(output_fname + ".model")

이젠 모델 평가...

In [32]:
from models.sent_eval import LDAEvaluator
model_eval = LDAEvaluator(output_fname)

/home/inhwan/Python/Pytorch_ev/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


show_topic_docs 함수에 토픽 ID를 인자로 주어 실행하면 해당 토픽 확률 값이 가장 높은 문서를 출력한다. 이 경우 슬픔과 주로 관련된 토픽인 것으로 보인다.

In [34]:
model_eval.show_topic_docs(topic_id=10)

[('누가 일용직으로도 먹고 살기 힘든 신용불량자 아버지, 고독하게 살다 죽음마저 쓸쓸한 어머니들, 그들을 이용해 홍보관으로 돈을 버는 악당아닌 악당을 만들었나... 부모란 존재의 책임감과 설움이 김인권님의 마지막 표정연기에 무너져내릴듯 먹먹하더라고요.',
  0.9833079),
 ('주인공이 겪어야 했던 심적 공허함이 보는내내 마음속 깊이 전해져왔고, 영화가 말해주는 전쟁의 참상에 눈물을 흘리지 않을 수 없었다. 현실적인 표현에 마치 내가 주인공의 상황이 된 듯한 착각마저 들게 하는 대단한 몰입력을 가진 영화이다.',
  0.9827287),
 ('김기덕 영화는 정말 보면 볼수록 이야기 설정이 기가 막힌 것 같다. 여러 갖가지 자신들의 주관에 의해 쓴 리뷰를 보고서 이런해석도 가능하고나 하고 생각하고 이해는 했으나, 실제 감독이 어떤 의도로 제작을 했는지가 더욱 알고 싶어졌다.',
  0.9824182),
 ('아들과 아버지의 재회와 생존 이후 아버지의 심경변화와 담배를 끝는 모습에서 감정이입되어서 정말 서럽게 울었고 마지막에 아버지가 구하러 나타났을 땐 통쾌하고 감동적이고 사랑마저 느껴졌다 구성도 완벽하고 설정하나만 가지고 스토리를 완벽하게 만들어냈다',
  0.982412),
 ('캐서린 비글로우는 여자지만 그 어떤 남자감독들보다도 더 리얼한 전쟁을 담았다. 인간이라면 두려워 하지 않을수 없는 폭탄해체를 담당하는 EOD팀이 폭탄을 해체하는 순간순간마다 공포영화보다 더 무서웠다. 제임스 중사를 맡은 제레미 레너의 연기도 훌륭했다.',
  0.9820576),
 ('슬프지만...잔잔한 여운도 남고...감동적이기도 하네요...영화를 보면서 잊었던 순간의 소중함을 깨닫게 되는거 같아요...언젠가 죽음의 순간이 올때 사랑하는 사람들이 곁에 있으면 그순간이 슬프지만은 않을꺼같아요...',
  0.98175615),
 ('영화내용은 사실별거 없지만 보고나면 생각을 많이하게 되는 영화임. 재미있기도 하고, 결론은 무엇이냐 하면 빨갱이들이 아무것도 모르는 사람들 선동해

토픽 ID에서 가장 높은 확률 값을 지니는 단어들 목록을 확인해보자. 대부분 조사여서, 피처를 만드는 과정에서 명사만 쓰기도 한다.

In [38]:
model_eval.show_topic_words(topic_id=10, topn=50)

[('는', 0.024055673),
 ('이', 0.023548668),
 ('다', 0.019944625),
 ('.', 0.019815419),
 ('영화', 0.017739704),
 ('고', 0.01522953),
 ('하', 0.014781788),
 ('에', 0.014623254),
 ('가', 0.014229408),
 ('들', 0.013348151),
 ('은', 0.0129511),
 ('를', 0.01272846),
 ('을', 0.012353053),
 ('게', 0.011946966),
 ('의', 0.01161769),
 ('었', 0.010754891),
 ('한', 0.010526416),
 ('도', 0.00994716),
 ('보', 0.008243409),
 ('적', 0.0076094694),
 (',', 0.00755657),
 ('있', 0.0074833934),
 ('과', 0.0064101447),
 ('지만', 0.0063548065),
 ('어', 0.006307504),
 ('것', 0.0063051237),
 ('없', 0.006225815),
 ('되', 0.006124328),
 ('지', 0.0059564635),
 ('에서', 0.0058481884),
 ('던', 0.0053997315),
 ('!', 0.0051406533),
 ('했', 0.0050991573),
 ('생각', 0.0050418302),
 ('감동', 0.004875533),
 ('기', 0.0043567945),
 ('보다', 0.004120134),
 ('내용', 0.004002017),
 ('좋', 0.0037932573),
 ('만', 0.0037327285),
 ('정말', 0.0036763763),
 ('연기', 0.0036569296),
 ('만들', 0.0035117327),
 ('는데', 0.00350418),
 ('사람', 0.0034370555),
 ('그', 0.0033457356),
 ('같', 0.00

새로운 문서의 토픽을 확인해보자. 문서를 형태소 분석한 뒤 이를 LDA 모델에 넣어 토픽을 추론해 가장 높은 확률 값을 지니는 토픽 ID와 그 확률을 리턴한다.

In [39]:
model_eval.show_new_document_topic(['너무 사랑스러운 영화','인생을 말하는 영화'])

너무 사랑스러운 영화 , topic id: 12 , prob: 0.8066549
인생을 말하는 영화 , topic id: 7 , prob: 0.7010173


In [40]:
model_eval.show_topic_docs(topic_id=12)

[('솔직히 생각보다 재밌어서 놀랐다... 이거 개봉하고 낸린줄도 모르고...ㅠ ㅠ 이제서야 봤네... 정말 두 사람의 커플댄스는... 명장면이다... 생각할수록 아쉽다... 단순 sm상업영화라고 생각하기엔... 두 사람의 춤이 너무 인상적이다...',
  0.97851247),
 ('오늘 재방으로 처음 봤는데 몰입도 상당했어요영화 화양연화 생각도 나구요감히 드러내기 힘든 그래서 더 아프고 조심스러운 어른들만의 사랑 혹은 불륜~ 좋아요',
  0.9745503),
 ('현실적이고 재밌게 꾸며주세요너무 막장은 싫고요인물들의 성격,행동이 일관됬으면 좋겠네요무조건 재미만 추구안했음 좋겠습니다',
  0.97237176),
 ('비밀을 내가 안봐서 비슷한건 모르겠고.. 그냥.. 태양은가득히 너무 재밌어요!! 전개도 빨라서 완전 흥미진진ㅋㅋㅋ 특히 윤계상씨 연기가 쩌네요',
  0.9715571),
 ('류승범, 공효진의 맛깔나는 연기. 임은경의 리즈 시절. 기타 단역 찾아보는 재미도(정우, 양익준, 이하늘). 상당히 매력적인 몇 안되는 한국 영화 중 하나.',
  0.97155094),
 ('바람의 파이터가실화여서 더 영감이간다여자하고도잘됬으면 그래도 배우들의 연기력도 좋고최배달이라는분에게 존경심을표합니다.',
  0.9706926),
 ('지상파에 나왔으면 대박났을텐데...진심 커플들 케미폭발 엄청나고 bgm도 드라마에 맞게 너무 좋고ㅠㅠ 초롱이 넘 이쁘다♡',
  0.96978134),
 ('예상치도 못한 사가 한국방영!이번 영화의 더빙에 참여해주신 모든 성우분들께 갈채를 보냅니다! 수고많으셨습니다!', 0.9677571),
 ('"넘 잼나서 아주 미치겄네♥♥ 싱싱♥고구마만 나왔음 좋겠어여...오세령은 삐@*:;""\'\';;///...ㅋ"',
  0.96663815),
 ('기대안하고 봤는데 단순 멜로물이 아니네ㅎㅎ 정말 세 커플의 이야기지만 결국은 하나의 이야기라는게 흥미진진했어요', 0.96546924)]

In [41]:
model_eval.show_topic_docs(topic_id=7)

[('일상적이면서도 감성적인 영화였다. 지구는 승차감이 없는 기차같다는 표현과 바다가 바람부는 냉장고 같다는 표현으로 시청각장애인들이 느끼는 세상을 잠시 훔쳐보고 온 느낌도 들었다. 아내 순호씨와의 애정이 느껴져 보는 내내 잔잔한 미소를 지었다.',
  0.98175114),
 ('문제아라고 쏘아부쳤던 아이들의 속내. 재빠르고 영리하고 의리있는 시릴 / 다르덴은 이렇게 엉망인 가운데에도 진심의 작은 마음이 지켜갈 수 있는 것에 대해 꾸준히 이야기해주네. 극적이지 않게 직조하는 능력에 감탄 ?@non',
  0.98173225),
 ('중학교 고등학교시절 세나와 프로스트 만쎌이 F1레이스를 다투던 기사를 보던게 생생하다.. 특히 멕라렌팀의 흰색과 빨강의 말보로 마킹이된 세나의 머신은 당시 최고로 빠르고 아름다운 아이콘으로 모터스포츠를 좋아하는 사람들의 동경대상이었다.. 아름다운 사람',
  0.9817193),
 ('장엄하고 감동적인 내인생 최고의 영화!명장면이 너무 많다. 라이온킹첫장면,beprepared,무파사죽는장면,죽은무파사가심바앞에영혼이 되어나타조언해주는 장면,하쿠나마타타,마지막심바vs스카대결씬까지 지금까지 가장 인상깊고감동적인최고의 영화',
  0.9813875),
 ('설란이가 세자에게 백허그하면서 노랠 부를때 눈물이 핑돌면서 따뜻한가슴을 느꼈다. 드라마가 끝난 지금도 여운이 길게 남으면서 그담회가 기다려진다. 수백향출연진 모두에게 박수를 보내드려요 화이팅!!!',
  0.97896886),
 ('처음봤을때는 졸리의 연기에 감탄했는데 그 영화를 또 보고 또 볼수록 위노나 라이더의 연기야 말로 진짜 대단하다는 걸 느끼게 됬다. 정말 그 주인공의 모든 감정들을 오버하지 않고 제대로 표현했다',
  0.9785137),
 ("영화보는내내 나를 보는 것 같아 소름끼쳤다 ..ㄷㄷ 미셸 공드리 식으로 '남자'에 대해 완전히 파헤친 영화!!강력 추천함 ㅎㅎㅎ 꼭 보시길(특히 짝사랑한 상대한테거절당한 남성에게 추천함)",
  0.97801894),
 ('지구의 종

뭔가 미묘한 거 같은데.... ㅋㅋㅋ